# Polygenic Score (PGS) Analysis – PGS001298

This notebook automates downloading, preprocessing, exploratory analysis, and visualization of the PGS001298 scoring file (GRCh38).

In [ ]:
!pip install pandas matplotlib --quiet

In [ ]:
import gzip
import shutil
import os
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlretrieve

In [ ]:
# Step 0: Download the PGS file
pgs_url = "https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS001298/ScoringFiles/Harmonized/PGS001298_hmPOS_GRCh38.txt.gz"
local_gz_file = "PGS001298_hmPOS_GRCh38.txt.gz"
if not os.path.exists(local_gz_file):
    print("Downloading PGS file...")
    urlretrieve(pgs_url, local_gz_file)
    print("Download complete!")
else:
    print("File already exists.")

In [ ]:
# Step 1: Unzip
def unzip_gz_file(gz_path):
    txt_path = gz_path.replace(".gz", "")
    if not os.path.exists(txt_path):
        with gzip.open(gz_path, "rb") as f_in:
            with open(txt_path, "wb") as f_out:
                shutil.copyfileobj(f_in, f_out)
    return txt_path

txt_file = unzip_gz_file(local_gz_file)
print(f"Unzipped file: {txt_file}")

In [ ]:
# Step 2: Preprocess PGS data
def preprocess_pgs_data(file_path):
    df = pd.read_csv(file_path, sep="\t", comment="#")
    df = df.dropna(axis=1, how="all")
    df = df.dropna()
    df["hm_chr"] = pd.to_numeric(df["hm_chr"], errors='coerce')
    df = df[df["hm_chr"].between(1, 22)]
    df["hm_chr"] = df["hm_chr"].astype(int)
    df["hm_pos"] = df["hm_pos"].astype(int)
    df["effect_weight"] = df["effect_weight"].astype(float)
    df["rsID"] = df["rsID"].astype(str)
    df = df.drop(columns=["chr_name", "chr_position"])
    output_file = "PGS001298_hmPOS_GRCh38_cleaned.txt"
    df.to_csv(output_file, sep="\t", index=False)
    print(f"Cleaned DataFrame saved to {output_file}")
    return df

clean_df = preprocess_pgs_data(txt_file)

In [ ]:
# Step 3: Exploratory analysis
def exploratory_analysis(clean_df):
    print("Number of variants:", clean_df.shape[0])
    print("Number of columns:", clean_df.shape[1])
    print("\nUnique chromosomes:", sorted(clean_df["hm_chr"].unique()))
    print("\nVariants per chromosome:\n", clean_df["hm_chr"].value_counts().sort_index())
    plt.figure(figsize=(6,4))
    plt.hist(clean_df["effect_weight"], bins=50)
    plt.xlabel("Effect Weight")
    plt.ylabel("Frequency")
    plt.title("Distribution of Effect Weight (All Chromosomes)")
    plt.show()

exploratory_analysis(clean_df)

In [ ]:
# Step 4: Chromosome 21 histogram
def plot_chr_effect_weight_histogram(clean_df, chr_num=21, save_path=None):
    chr_df = clean_df[clean_df["hm_chr"] == chr_num]
    if chr_df.empty:
        print(f"No variants found for chromosome {chr_num}.")
        return
    plt.figure(figsize=(6,4))
    plt.hist(chr_df["effect_weight"], bins=50)
    plt.xlabel("Effect Weight")
    plt.ylabel("Frequency")
    plt.title(f"Effect Weight Distribution on Chromosome {chr_num}")
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    plt.show()
    print(f"Number of variants on chromosome {chr_num}:", chr_df.shape[0])
    print(chr_df["effect_weight"].describe())

plot_chr_effect_weight_histogram(clean_df, save_path="chr21_effect_weight_hist.png")